# Term Frecuency x Inverse Document Frecuency (TF IDF)


In [1]:
import io
import re
from functools import reduce
from collections import Counter
from numpy import log
from pandas import DataFrame

In [2]:
with io.open('stop-words-spanish.txt', mode='r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()
# stopwords

In [3]:
# AMLO_28042013
with io.open('Data/AMLO_20180429.txt', mode='r', encoding='utf-8') as f:
    boletin = f.read().splitlines()
boletin

['Palenque, Chiapas, 29 de abril de 2018.- Andrés Manuel López Obrador lamentó las situaciones en las que están el candidato del PRI, José Antonio Meade, porque “no quisiera estar en sus zapatos” y el candidato del PAN, Ricardo Anaya, así como de sus patrocinadores y de los intelectuales alcahuetes que lo apoyan.',
 'A los apoyadores e intelectuales orgánicos al servicio de la mafia del poder que ayudan a Anaya, López Obrador les dijo que lo que no pueden hacer en política es el ridículo.',
 'A la pregunta de los reporteros sobre que el EZLN dice que los poderes económicos no los dejará llegar, López Obrador respondió que no, no, porque las botargas y los mensajes, dan hasta si no sueño, “son ternuritas”.',
 'Sobre que hace rato MORENA en Jalapa acusó al gobernador de Veracruz, Miguel Ángel Yunes de poner unas botargas de Hugo Chávez en el inicio de la campaña de Cuitláhuac García, el candidato a la Presidencia de México sostuvo que no va a pasar nada, están muy nerviosos.',
 'Recordó 

### Obtener palabras

Obtenemos las palabras en minúsculas


In [4]:
documentos = [re.findall('[a-zA-zñáéíóúüÁÉÍÓÚÜÑ]+', parrafo.lower()) for (index, parrafo) in enumerate(boletin)]
# documentos

Obtenemos documentos sin stopwords

In [5]:
documentos = [[palabra for palabra in documento if not (palabra in stopwords)] for documento in documentos]
total_palabras = len(documentos)
# documentos

Contar las palabras por documento y el total de palabras

In [6]:
cuenta_por_documento = [Counter(parrafo) for parrafo in documentos]
cuenta_total_palabras = reduce((lambda x, y: x + y), cuenta_por_documento)
palabras = [palabra for (palabra, cuenta) in cuenta_total_palabras.items()]
# cuenta_por_documento

Definimos función para suavizar la cuenta de palabras.

In [7]:
def bm25(contador, k=25):
    return {
        key: ((cuenta + 1)*k)/(cuenta+k) for (key, cuenta) in contador.items()
    }

Calculamos tf para cada uno de los documentos 

In [8]:
tfs = [bm25(c_doc) for c_doc in cuenta_por_documento]
# tfs

Definimos la función IDF

In [9]:
def idf(palabra, documentos):
    return log(
        len(documentos) / 
        (1+reduce(lambda cuenta, documento : cuenta + (1 if (palabra in documento) else 0), documentos, 0))
    )
# documento for documento in documentos if (palabra in documento)
# ]))

Calculamos el idf de todas las palabras

In [10]:
idfs = {
    palabra: idf(palabra, documentos) for palabra in palabras
}
# idfs

Calculamos tfxIDF

In [11]:
tfidfs = [
    {
        palabra: idfs[palabra]*tf
        for (palabra, tf) in tf_n.items()
    }
    for tf_n in tfs
]
# tfidfs

In [12]:
def obtener_keywords(num_keywords=5):
    keywords = []
    for tfidf_n in tfidfs:
        keywords_n = sorted(tfidf_n.items() ,  key=lambda x: x[1], reverse=True)[:num_keywords]
#         print([keyword[0] for keyword in keywords_n])
        keywords.append([keyword[0] for keyword in keywords_n])
    return keywords

In [13]:
keywords = obtener_keywords(6)
# keywords

### Palabras clave por párrafo

In [14]:
keywords_dict = {'Párrafo {0}'.format(n+1):kw_n for (n, kw_n) in enumerate(keywords)}
df = DataFrame(keywords_dict).transpose()
df

,0,1,2,3,4,5
Párrafo 1,abril,lamentó,situaciones,josé,antonio,meade
Párrafo 2,apoyadores,orgánicos,servicio,mafia,poder,ayudan
Párrafo 3,ezln,poderes,económicos,dejará,llegar,mensajes
Párrafo 4,hace,rato,jalapa,acusó,miguel,ángel
Párrafo 5,recordó,publicó,cuenta,facebook,siguen,seguir
Párrafo 6,convocó,digan,pagó,comprometió,transmito,feis
Párrafo 7,millones,pesos,sabe,dispone,transmita,asunto
Párrafo 8,habla,belaunzarán,graco,denise,dresser,cerca
Párrafo 9,acepto,debate,intercambiar,puntos,vista,cara
Párrafo 10,directores,dicen,presionando,bodrio,asqueroso,fascistoide


### Palabras más usadas

In [15]:
DataFrame(sorted(cuenta_total_palabras.items(),  key=lambda x: x[1], reverse=True)[:15], columns=['Palabra', 'Cuenta'])

,Palabra,Cuenta
0,chiapas,16
1,obrador,15
2,lópez,14
3,candidato,8
4,palenque,7
5,méxico,7
6,gobierno,7
7,historia,6
8,manuel,5
9,dijo,5
